# PhoneLM

## Test `G2P` and `Encodec`

In [ ]:
!pip install g2p_en encodec

### `G2P`

In [2]:
from g2p_en import G2p

In [3]:
import torch
import random
import string
from functools import cache
from tqdm import tqdm

@cache
def _get_model():
    return G2p()

@cache
def _get_graphs(path):
    with open(path, "r") as f:
        graphs = f.read()
    return graphs

def encode(graphs: str) -> list[str]:
    g2p = _get_model()
    phones = g2p(graphs)
    ignored = {" ", *string.punctuation}
    return ["_" if p in ignored else p for p in phones]

@torch.no_grad()
def write_phones(folder, suffix=".normalized.txt"):
    print("ello?")
    paths = list(folder.rglob(f"*{suffix}"))
    random.shuffle(paths)

    print("paths:", paths)
    for path in tqdm(paths):
        phone_path = path.with_name(path.stem.split(".")[0] + ".phn.txt")
        if phone_path.exists():
            continue
        print("?")
        graphs = _get_graphs(path)
        phones = encode(graphs)
        with open(phone_path, "w") as f:
            f.write(" ".join(phones))

In [4]:
from pathlib import Path
write_phones(Path("./data/text"))

ello?
paths: [WindowsPath('data/text/test.normalized.txt')]


100%|██████████| 1/1 [00:00<?, ?it/s]


### `Encodec`

In [2]:
from tqdm import tqdm
import random
import torch
from functools import cache
import torchaudio
from encodec import EncodecModel
from torch import Tensor
from einops import rearrange
import soundfile
from encodec.utils import convert_audio
from pathlib import Path

SAMPLE_RATE = 24_000
BANDWIDTHS  = [1.5, 3.0, 6.0, 12.0, 24.0]
BANDWIDTH   = BANDWIDTHS[0]

@cache
def _load_model(bandwidth=6.0, device="cuda"):
    # Instantiate a pretrained EnCodec model
    assert SAMPLE_RATE == 24_000
    model = EncodecModel.encodec_model_24khz()
    model.set_target_bandwidth(bandwidth)
    model.to(device)
    return model

def unload_model():
    return _load_model.cache_clear()

@torch.inference_mode()
def decode(codes: Tensor, bandwidth=6.0, device="cuda"):
    """
    Args:
        codes: (b q t)
    """
    assert codes.dim() == 3
    model = _load_model(bandwidth, device)
    return model.decode([(codes, None)]), model.sample_rate

def decode_to_file(resps: Tensor, path: Path):
    assert resps.dim() == 2, f"Require shape (t q), but got {resps.shape}."
    resps = rearrange(resps, "t q -> 1 q t")
    wavs, sr = decode(codes=resps, bandwidth=BANDWIDTH)
    soundfile.write(str(path), wavs.cpu()[0, 0], sr)

def _replace_file_extension(path, suffix):
    return (path.parent / path.name.split(".")[0]).with_suffix(suffix)

@torch.inference_mode()
def encode(wav: Tensor, sr: int, bandwidth=6.0, device="cuda"):
    """
    Args:
        wav: (t)
        sr: int
    """
    model = _load_model(bandwidth, device)
    wav = wav.unsqueeze(0)
    wav = convert_audio(wav, sr, model.sample_rate, model.channels)
    wav = wav.to(device)
    encoded_frames = model.encode(wav)
    qnt = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)  # (b q t)
    return qnt

def encode_from_file(path, bandwidth=6.0, device="cuda"):
    wav, sr = torchaudio.load(str(path))
    if wav.shape[0] == 2:
        wav = wav[:1]
    return encode(wav, sr, bandwidth, device)

def quantize_audio(folder, suffix=".wav"):
    paths = [*folder.rglob(f"*{suffix}")]
    random.shuffle(paths)

    for path in tqdm(paths):
        out_path = _replace_file_extension(path, ".qnt.pt")
        if out_path.exists():
            continue
        qnt = encode_from_file(path, BANDWIDTH)
        print(qnt.shape)
        torch.save(qnt.cpu(), out_path)

def decode_files(folder, suffix=".qnt.pt"):
    paths = [*folder.rglob(f"*{suffix}")]
    random.shuffle(paths)

    for path in tqdm(paths):
        out_path = _replace_file_extension(path, ".qt.wav")
        if out_path.exists():
            continue
        fi = rearrange(torch.load(path).squeeze(0).cuda(), "q t -> t q")
        decode_to_file(fi, out_path)

In [3]:
# from pathlib import Path
# quantize_audio(Path("./data/audio"))
# decode_files(Path("./data/audio"))

In [4]:
# torch.load("data/audio/test.qnt.pt").shape

#### Generate Audio from Tensor

In [9]:
audio_tensor = torch.tensor([[1019,  662],
        [ 598,   25],
        [ 321,  463],
        [1063,  575],
        [ 745,  727],
        [1073,  344],
        [1098,  344],
        [1046,  959],
        [1062,  874],
        [1059,  804],
        [1038, 1010],
        [1081,  577],
        [1098,  323],
        [1049,  858],
        [1034,  278],
        [1098,  469],
        [1069,  626],
        [1034,  482],
        [1071,  398],
        [1063,  858],
        [1083,  443],
        [1034,  418],
        [1072,  632],
        [1075,  914],
        [1098, 1010],
        [1094,  357],
        [1087,  898],
        [1084,  702],
        [1099,  654],
        [ 835,  364],
        [ 208,  416],
        [ 987,  722],
        [ 872,  708],
        [ 994,  399],
        [ 264,  648],
        [ 264, 1007],
        [1001,  961],
        [ 598,  320],
        [ 360,  993],
        [ 879,  747],
        [ 325,  700],
        [  52,  770],
        [ 257,  268],
        [ 257,  824],
        [ 819,  662],
        [ 709,  567],
        [ 656,  662],
        [  43,  602],
        [1038,  742],
        [  24,  964],
        [1098,  289],
        [1099,  722],
        [ 855,  870],
        [  25,  561],
        [ 472,  519],
        [ 472,  754],
        [ 475, 1038],
        [ 404,  857],
        [ 331,  913],
        [ 574,  434],
        [ 537,  154],
        [1022,  612],
        [ 324,  321],
        [ 937,  563],
        [ 230, 1001],
        [ 912,  563],
        [ 912,  807],
        [ 928,   99],
        [ 928,   99],
        [ 942,  228],
        [ 604,  772],
        [ 904,   94],
        [ 472, 1063],
        [  52,  812],
        [  52,  645],
        [  52,  697],
        [ 257,  387],
        [  52,  362],
        [ 935,  247],
        [ 983,   65],
        [ 683,  874],
        [ 155,  518],
        [  30,  822],
        [ 855,  467],
        [ 904,  909],
        [ 904,  529],
        [ 904,  852],
        [ 855,  399],
        [ 855,  470],
        [ 855, 1023],
        [ 106,  870],
        [ 176,  580],
        [ 574,  669],
        [ 502,  888],
        [ 588,  708],
        [ 782,  700],
        [ 588,  743],
        [ 890,  417],
        [ 373,  822],
        [ 160,  514],
        [  47,  455],
        [  47,  328],
        [  47,  259],
        [ 909,  971],
        [1023,  962],
        [ 577,  367]]).cuda()

In [10]:
audio_tensor = torch.clamp(audio_tensor, min=0, max=1023)

In [11]:
audio_tensor.shape

torch.Size([106, 2])

In [12]:
decode_to_file(audio_tensor, "general_out.wav")

## Dataset

### LJSpeech

In [1]:
BANDWIDTH_IDX = 1 # original VALL-E
CODEBOOKS     = [2, 4, 8, 16, 32]
BANDWIDTHS    = [1.5, 3.0, 6.0, 12.0, 24.0]
BANDWIDTH     = BANDWIDTHS[BANDWIDTH_IDX]
CODEBOOK      = CODEBOOKS[BANDWIDTH_IDX]

import torchaudio
from ljspeech import LJSPEECH
DATASET_PATH = "./data/LJSpeech/"
dataset = LJSPEECH(
    "./data/LJSpeech",
    encodec_bandwidth=BANDWIDTH)

In [2]:
len(dataset)

1919

In [3]:
dataset[0][-1].shape

torch.Size([1, 4, 143])

In [4]:
import torch
import torchaudio
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split

indices = list(range(len(dataset)))
train_indices, test_indices = train_test_split(indices, test_size=0.1, random_state=42)

train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler, collate_fn=lambda x: x)
test_loader = DataLoader(dataset, batch_size=32, sampler=test_sampler, collate_fn=lambda x: x)

In [5]:
len(train_loader), len(test_loader)

(54, 6)

In [6]:
item = next(iter(train_loader))

In [7]:
item

[(WindowsPath('data/LJSpeech/LJSpeech-1.1/wavs/LJ003-0134.wav'),
  tensor([[ 0.0004,  0.0004,  0.0002,  ..., -0.0006, -0.0004, -0.0005]]),
  22050,
  'One day he was too ill to come down and meet her.',
  'One day he was too ill to come down and meet her.',
  ['W',
   'AH1',
   'N',
   '_',
   'D',
   'EY1',
   '_',
   'HH',
   'IY1',
   '_',
   'W',
   'AA1',
   'Z',
   '_',
   'T',
   'UW1',
   '_',
   'IH1',
   'L',
   '_',
   'T',
   'UW1',
   '_',
   'K',
   'AH1',
   'M',
   '_',
   'D',
   'AW1',
   'N',
   '_',
   'AH0',
   'N',
   'D',
   '_',
   'M',
   'IY1',
   'T',
   '_',
   'HH',
   'ER0',
   '_',
   '_'],
  tensor([70, 11, 48, 74, 24, 33, 74, 37, 42, 74, 70,  5, 72, 74, 60, 67, 74, 39,
          46, 74, 60, 67, 74, 45, 11, 47, 74, 24, 17, 48, 74, 10, 48, 24, 74, 47,
          42, 60, 74, 37, 29, 74, 74], device='cuda:0'),
  tensor([[[1019,  942,  197,  796,  730,  556,  886,  997,  790,  790,  255,
             255, 1008,  182,  862,  687,  860,  753,   52,  935,  125, 

## Model

In [8]:
import megabyte
import torch
import torch.nn as nn
from einops import rearrange

def get_reserved_mem_gb():
    device = torch.cuda.current_device()
    reserved = torch.cuda.memory_reserved(device)
    reserved_gb = reserved / 1024 / 1024 / 1024
    return reserved_gb

class PhoneLM(nn.Module):
    def __init__(self, n_phone_tokens, n_audio_tokens):
        super(PhoneLM, self).__init__()
        self.megabyte   = megabyte.MEGABYTE(
            heads       = 8, # 1,
            dim_head    = 32, # 16,
            num_tokens  = n_phone_tokens + n_audio_tokens + 4,
            dim         = (768, 256, 128), # (32, 32, 32), # (768, 256, 128)# Dg, Dl1, Dl2
            depth       = (6, 4, 2), # (6, 4, 2)
            max_seq_len = (32, 4, 4), # (128, 4, 4), # , # 512
            flash_attn  = False)

    def forward(self, x, debug=False, return_loss=True):
        x = self.megabyte(x, return_loss=return_loss)
        return x
    
    def get_params(self):
        o = [param.numel() for param in self.parameters() if param.requires_grad]
        o = sum(o)
        return o
    
    def generate(self, *args):
        return self.megabyte.generate(*args)
    
def multi_encode(
        phone_tokens,
        audio_tokens,
        n_phone_tokens,
        n_audio_tokens,
        max_clip_length=1.0):
    """NOTE: 75 steps per second for 24kHz in `encodec.
    Set `max_clip_length` to 0 for original clip length."""

    # Start text token, end text token, start audio token, end audio token
    ETT, EAT = [n_phone_tokens + n_audio_tokens + i
                          for i in range(2)]
    ETT = torch.tensor([ETT]).long().cuda()
    EAT = torch.tensor([EAT]).long().cuda()

    if max_clip_length:
        print("pre audio_tokens.shape", audio_tokens.shape)
        audio_tokens = audio_tokens[:, :, :int(max_clip_length * 75)]
    audio_tokens = rearrange(audio_tokens.squeeze(0), "q s -> (q s)")
    print("post audio_tokens.shape", audio_tokens.shape)
    
    # offset phone tokens past audio tokens
    phone_tokens += n_audio_tokens
    
    print("phone_tokens.shape:", phone_tokens.shape)
    print("audio_tokens.shape:", audio_tokens.shape)
    
    device = torch.cuda.current_device()
    phone_tokens = torch.cat((phone_tokens, ETT), dim=0).to(device)
    audio_tokens = torch.cat((audio_tokens, EAT,), dim=0).to(device)
    combined_tokens = torch.cat((phone_tokens, audio_tokens), dim=0).to(device)
    return phone_tokens, audio_tokens, combined_tokens

In [9]:
from einops import rearrange

from encodec_util import decode_to_file

"""
EinopsError:  Error while processing rearrange-reduction pattern "(t q) -> t q".
 Input tensor shape: torch.Size([75]). Additional info: {'q': 4, 't': 75}.
 Shape mismatch, 75 != 300
"""

def generate_audio(sample,
                   n_phone_tokens,
                   n_audio_tokens,
                   audio_path="./out.wav"):
    ETT, EAT = [n_phone_tokens + n_audio_tokens + i
                          for i in range(2)]
    ST_S = [ETT, EAT]
    print("ETT, EAT ids:", ST_S)
    seq = sample.cpu().tolist()[0]
    print("seq:", seq)
    # all special tokens in list
    if all(st_t in seq for st_t in ST_S) and len(seq) >= len(ST_S) + 2:
        # text_tokens  = seq[seq.index(STT + 1):seq.index(ETT - 1)]
        audio_tokens = seq[seq.index(ETT)+1:seq.index(EAT)]
        print(seq.index(ETT), seq.index(EAT), len(audio_tokens))
        audio_tokens = torch.tensor(audio_tokens).cuda()
        audio_tokens = rearrange(
            audio_tokens,
            '(t q) -> t q',
            q=1, # CODEBOOK,
            t=audio_tokens.size(0) // 1) # t=audio_tokens.size(0) // CODEBOOK)
        print("audio_tokens.shape:", audio_tokens, audio_tokens.shape)
        decode_to_file(audio_tokens, audio_path)
        return True
    else:
        return False

## PhoneLM - LJSpeech

### Train

In [10]:
device  = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PhoneLM(
    n_phone_tokens=len(dataset.phone_dict),
    n_audio_tokens=1024).to(device)

model.megabyte.get_num_params()

number of parameters: 37.30M


37302863

In [11]:
item = next(iter(train_loader))[0]
item_phone_tokens = item[-2]
# item_audio_tokens = item[-1]
item_audio_tokens = item[-1][:, 0, :] # Only keep primary coarse tokens, for now
item_audio_tokens = item_audio_tokens.unsqueeze(0)
item_phone_tokens.shape, item_audio_tokens.shape

(torch.Size([54]), torch.Size([1, 1, 257]))

In [12]:
item[3]

'as to the fact that he was an outstanding man, end quote.'

In [13]:
item

(WindowsPath('data/LJSpeech/LJSpeech-1.1/wavs/LJ045-0092.wav'),
 tensor([[0.0006, 0.0008, 0.0007,  ..., 0.0008, 0.0007, 0.0007]]),
 22050,
 'as to the fact that he was an outstanding man, end quote.',
 'as to the fact that he was an outstanding man, end quote.',
 ['AE1',
  'Z',
  '_',
  'T',
  'UW1',
  '_',
  'DH',
  'AH0',
  '_',
  'F',
  'AE1',
  'K',
  'T',
  '_',
  'DH',
  'AE1',
  'T',
  '_',
  'HH',
  'IY1',
  '_',
  'W',
  'AA1',
  'Z',
  '_',
  'AE1',
  'N',
  '_',
  'AW2',
  'T',
  'S',
  'T',
  'AE1',
  'N',
  'D',
  'IH0',
  'NG',
  '_',
  'M',
  'AE1',
  'N',
  '_',
  '_',
  '_',
  'EH1',
  'N',
  'D',
  '_',
  'K',
  'W',
  'OW1',
  'T',
  '_',
  '_'],
 tensor([ 8, 72, 74, 60, 67, 74, 25, 10, 74, 35,  8, 45, 60, 74, 25,  8, 60, 74,
         37, 42, 74, 70,  5, 72, 74,  8, 48, 74, 18, 60, 58, 60,  8, 48, 24, 38,
         49, 74, 47,  8, 48, 74, 74, 74, 27, 48, 24, 74, 45, 70, 51, 60, 74, 74],
        device='cuda:0'),
 tensor([[[738, 275, 747,  ..., 738, 855, 738],
        

In [14]:
phone_prompt, audio_target, test_inp = multi_encode(
    item_phone_tokens,
    item_audio_tokens,
    n_phone_tokens=len(dataset.phone_dict),
    n_audio_tokens=1024,
    max_clip_length=5)
test_inp.shape

pre audio_tokens.shape torch.Size([1, 1, 257])
post audio_tokens.shape torch.Size([257])
phone_tokens.shape: torch.Size([54])
audio_tokens.shape: torch.Size([257])


torch.Size([313])

### Training Process

In [15]:
from tqdm.notebook import tqdm

In [16]:
import torch.optim as optim

epochs = 10

MAX_LR       = 1e-2
# MAX_LR       = 1e-2
WEIGHT_DECAY = 1e-4
GRAD_CLIP    = 0.1

optimizer = optim.Adam(
    model.parameters(),
    lr=MAX_LR)
    #,weight_decay=WEIGHT_DECAY)

# def get_lr(optimizer):
#     for param_group in optimizer.param_groups:
#         return param_group['lr']

# sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, MAX_LR, epochs=epochs, 
#                                                 steps_per_epoch=len(trainloader))

In [17]:
test_inp.dtype

torch.int64

In [18]:
test_inp

tensor([1032, 1096, 1098, 1084, 1091, 1098, 1049, 1034, 1098, 1059, 1032, 1069,
        1084, 1098, 1049, 1032, 1084, 1098, 1061, 1066, 1098, 1094, 1029, 1096,
        1098, 1032, 1072, 1098, 1042, 1084, 1082, 1084, 1032, 1072, 1048, 1062,
        1073, 1098, 1071, 1032, 1072, 1098, 1098, 1098, 1051, 1072, 1048, 1098,
        1069, 1094, 1075, 1084, 1098, 1098, 1099,  738,  275,  747,  941,  941,
         146,   65,  812,  830,  807,  988,  744,   47,  934,   73,  323,  370,
         488,  619,  574,  676,  446,  319,  765,  860,  472,  676,  385,  755,
         414,  807,  339,  148,  237,  488,  798,  488,  176,  395,  339,  709,
         986,  903,  222,  248,  240,  216,  593,  645,  754,  901,  275,  257,
         370,  370,  257,  257,  904,  257,  709,  914,  914,  994,  916,  339,
         148,  148,  276,   25,   25,  136,  698,  549,  985,  465,  344,  457,
         583,  699,  583,  942,  656,  749,  994,  994,  881,  574,  934,  934,
         934,  881,  958,  694,  523,  8

In [19]:
test_inp.dtype

torch.int64

In [20]:
test_inp.shape

torch.Size([313])

In [21]:
import torch.nn.functional as F

EPOCHS = 1000
PRINT_INTERVAL = 100

seq_len = 512 # 2048

def train(model, trainloader):
    model.train()
    
    padding_len = max(0, seq_len - test_inp.size(0))
    n_test_inp = F.pad(test_inp, (0, padding_len))
    batch = n_test_inp.unsqueeze(0)
    # print(batch.shape)
    loss = model(batch, return_loss=True)
    # loss = model(next(trainloader), return_loss=True)
    loss.backward()
    return loss

# pbar = tqdm.tqdm(EPOCHS, mininterval=10., desc='training')
for epoch in range(EPOCHS):
    loss = train(model, train_loader)
    optimizer.step()
    optimizer.zero_grad()
    mem_gb = get_reserved_mem_gb()
    if epoch % PRINT_INTERVAL == 0:
        print(f"Reserved Memory (GB): {mem_gb}, loss: {loss.item()}")
    #' pbar.set_description(f"Reserved Memory (GB): {mem_gb}, loss: {loss.item()}")

Reserved Memory (GB): 1.005859375, loss: 7.007339954376221
Reserved Memory (GB): 1.005859375, loss: 0.03658662736415863
Reserved Memory (GB): 1.005859375, loss: 0.009765789844095707
Reserved Memory (GB): 1.005859375, loss: 0.00932304747402668
Reserved Memory (GB): 1.005859375, loss: 0.009168551303446293
Reserved Memory (GB): 1.005859375, loss: 0.009082884527742863
Reserved Memory (GB): 1.005859375, loss: 0.009153591468930244
Reserved Memory (GB): 1.005859375, loss: 0.008994382806122303
Reserved Memory (GB): 1.005859375, loss: 0.008969676680862904
Reserved Memory (GB): 1.005859375, loss: 0.008951202034950256


### Evaluate

In [29]:
phone_prompt.shape

torch.Size([55])

In [30]:
audio_target

tensor([ 738,  275,  747,  941,  941,  146,   65,  812,  830,  807,  988,  744,
          47,  934,   73,  323,  370,  488,  619,  574,  676,  446,  319,  765,
         860,  472,  676,  385,  755,  414,  807,  339,  148,  237,  488,  798,
         488,  176,  395,  339,  709,  986,  903,  222,  248,  240,  216,  593,
         645,  754,  901,  275,  257,  370,  370,  257,  257,  904,  257,  709,
         914,  914,  994,  916,  339,  148,  148,  276,   25,   25,  136,  698,
         549,  985,  465,  344,  457,  583,  699,  583,  942,  656,  749,  994,
         994,  881,  574,  934,  934,  934,  881,  958,  694,  523,  860,  598,
         224,  327,  388,  372,  804,  645,  185,  582,  613,  901,  185,  185,
         879,  257,  257,  904,  395,   47,   47,   47,   47,   47,   47,   47,
         160,  876,  709,  709,  463,  496,  385,   61,  790,  230,  222,  230,
         730,  361,  230,  822,  650, 1010,  344,  583, 1010,  976,  496,  456,
         327,  961,  906,  151,  945,  5

In [31]:
def generate(model, prompt):
    model.eval()

    prompt = prompt.unsqueeze(0)
    sample = model.generate(prompt)
    sample = sample.flatten(1)
    print("sample:", sample, sample.shape)

    return prompt, sample

prompt, sample = generate(model, phone_prompt)

100%|██████████| 457/457 [00:06<00:00, 75.15it/s]

sample: tensor([[1032, 1096, 1098, 1084, 1091, 1098, 1049, 1034, 1098, 1059, 1032, 1069,
         1084, 1098, 1049, 1032, 1084, 1098, 1061, 1066, 1098, 1094, 1029, 1096,
         1098, 1032, 1072, 1098, 1042, 1084, 1082, 1084, 1032, 1072, 1048, 1062,
         1073, 1098, 1071, 1032, 1072, 1098, 1098, 1098, 1051, 1072, 1048, 1098,
         1069, 1094, 1075, 1084, 1098, 1098, 1099,  738,  275,  747,  941,  941,
          146,   65,  812,  830,  807,  988,  744,   47,  934,   73,  323,  370,
          488,  619,  574,  676,  446,  319,  765,  860,  472,  676,  385,  755,
          414,  807,  339,  148,  237,  488,  798,  488,  176,  395,  339,  709,
          986,  903,  222,  248,  240,  216,  593,  645,  754,  901,  275,  257,
          370,  370,  257,  257,  904,  257,  709,  914,  914,  994,  916,  339,
          148,  148,  276,   25,   25,  136,  698,  549,  985,  465,  344,  457,
          583,  699,  583,  942,  656,  749,  994,  994,  881,  574,  934,  934,
          934,  881,

In [32]:
sample.shape

torch.Size([1, 512])

In [33]:
ETT, EAT = [len(dataset.phone_dict) + 1024 + i
                          for i in range(2)]
# sample.index(STT)

In [34]:
out = generate_audio(
    sample,
    n_phone_tokens=len(dataset.phone_dict),
    n_audio_tokens=1024)

ETT, EAT ids: [1099, 1100]
seq: [1032, 1096, 1098, 1084, 1091, 1098, 1049, 1034, 1098, 1059, 1032, 1069, 1084, 1098, 1049, 1032, 1084, 1098, 1061, 1066, 1098, 1094, 1029, 1096, 1098, 1032, 1072, 1098, 1042, 1084, 1082, 1084, 1032, 1072, 1048, 1062, 1073, 1098, 1071, 1032, 1072, 1098, 1098, 1098, 1051, 1072, 1048, 1098, 1069, 1094, 1075, 1084, 1098, 1098, 1099, 738, 275, 747, 941, 941, 146, 65, 812, 830, 807, 988, 744, 47, 934, 73, 323, 370, 488, 619, 574, 676, 446, 319, 765, 860, 472, 676, 385, 755, 414, 807, 339, 148, 237, 488, 798, 488, 176, 395, 339, 709, 986, 903, 222, 248, 240, 216, 593, 645, 754, 901, 275, 257, 370, 370, 257, 257, 904, 257, 709, 914, 914, 994, 916, 339, 148, 148, 276, 25, 25, 136, 698, 549, 985, 465, 344, 457, 583, 699, 583, 942, 656, 749, 994, 994, 881, 574, 934, 934, 934, 881, 958, 694, 523, 860, 598, 224, 327, 388, 372, 804, 645, 185, 582, 613, 901, 185, 185, 879, 257, 257, 904, 395, 47, 47, 47, 47, 47, 47, 47, 160, 876, 709, 709, 463, 496, 385, 61, 790, 230, 

In [35]:
out

True